In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


In [2]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [3]:
mnist_dataset

{'test': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'train': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}

In [4]:
mnist_train=mnist_dataset['train']
mnist_test=mnist_dataset['test']

In [5]:
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)
num_test_samples=mnist_info.splits['train'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image=tf.cast(image,tf.float32)
    image=image/255.#dot. we want the reult to be in float
    return image,label
scaled_train_and_validation_data=mnist_train.map(scale)

test_data=mnist_test.map(scale)


#shuffling , so there is no oredred data
#when we are deling wwith enormous datasets, we cant shyffle all data at once.

BUFFER_SIZE=10000# in this way we incerase computional power

#Shuffle at a batch of 10000,if buffer size=1, no shuffling will happen

shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

#ACTUALLY EXTRACT TRAIN AND VALIDATION DATA

validation_data=shuffled_train_and_validation_data.take(num_validation_samples)#to extract given samples samples
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)#skip first entered samples

#we will use many batches,have trade off between speed and accuracy 
#batch size=1==>SGD
#batch size =#samples==>single batch ,GD
#1<batch size<#samples==>mini batch GD==> we will use this


BATCH_SIZE=100

#dataset.batch(batch_size)=>a method that combines the consecutve elements of a datset into batches

train_data=train_data.batch(BATCH_SIZE)#IT WILL ADD NEW COLUMN TO OUR TENSOR
#THAT WILL INDICATE HOW MANY SAMPLES SHOULD WE TAKE IN EACH BATCH

# we dont need to batch on validation data,as we dont need to back prpogate onn validation data but only forwrd propgating
#batching was used  when updating weights only once per batch i.e. 100 samples rather than on every samples.hence reducing
#noise in traning updates. when batching we find the averge loss and average accuracy.during  validation and testing we want 
#exact values  so taking all data once.while forward propagating  we dont neeed much computational power so no batching required.
#however the model expects the validation datset inn batch form too that why we overwrite validation data

validation_data=validation_data.batch(num_validation_samples)#here we will have single batch with batch size=total number of samples
#all data size at once

test_data=test_data.batch(num_test_samples)

validation_inputs,validation_targets=next(iter(validation_data))
#iter() creates  an object which can be iterated one element at a time (e.g. in a for loop or while loop),pythonic
#synatx to make validation_data as an iterable
#next() loads the next element of an iteratble object in this case it is batch, as there is one batch it will load inputs and 
#targets

In [6]:
# outline the model
#input layer=784,2 hidden layer with units 50 each,output layer=10
#try fine tune hyperparametes that is width and depth
input_size=784
output_size=10
hidden_layer_size=50#assuming each hidden layer has same no of units
model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),#relu=rectified linear unit
    tf.keras.layers.Dense(output_size,activation='softmax')#it gives the probability of outputs
    
    
    
])
#our data is (from tfds) is such that each input is 28*28*1 or tensor of rank 3

#tf.keras.layers.Flatten(orignal shape)=>transforms or flattens a tensor into a vector
#we need to flatten the images into vectors


In [7]:
#opyimizer and loss function
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#Binary_crossentropy->where we have got binary coding,categorical_crossentropy->
#expects that we have one hot encoded the targets
#sparse_categorical_crossentropy-->applies one hot encoding





In [8]:
"""
training
steps
1>at the begining of each epoch, the training loss will be set to 0.
2>the algorithm will iterate over a present number of batches,all from train data.
3> the weights and bises will be updated as many times as there are batches.
4> we will get a value for the loss function, indicating how the training is going.
5>we will aslo see a tarining accuracy.
6> at the end of the epoch, the algo will forward propagate the whole  validation set
*when we reach the maximum number of epochs the training  will be over

the accuracy shows in what percent of the cases our outputs were equal to the targets
we keep a eye over validation loss (or set early stopping machanics) to determine whether the model is overfittig
val_accuracy=true accuracy of model
train accuracy is accuracy of batch.

The error happens because a tf.Dataset is provided to the argument validation_data of Model.fit,
but Keras does not know how many steps to validate for.
To solve this problem, you can just set the argument validation_steps. For example:
"""

NUM_EPOCHS=5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2,validation_steps=num_validation_samples)

Epoch 1/5
540/540 - 20s - loss: 0.4145 - accuracy: 0.8822 - val_loss: 0.2343 - val_accuracy: 0.9335
Epoch 2/5
540/540 - 15s - loss: 0.1846 - accuracy: 0.9465 - val_loss: 0.1768 - val_accuracy: 0.9463
Epoch 3/5
540/540 - 17s - loss: 0.1393 - accuracy: 0.9593 - val_loss: 0.1434 - val_accuracy: 0.9562
Epoch 4/5
540/540 - 15s - loss: 0.1147 - accuracy: 0.9664 - val_loss: 0.1206 - val_accuracy: 0.9650
Epoch 5/5
540/540 - 14s - loss: 0.0976 - accuracy: 0.9710 - val_loss: 0.1102 - val_accuracy: 0.9655


In [9]:
"""
test the model
play with various hyperparameteres,
crate a model--> fiddle with hyperparameters--> check validation acuracy
we might overfit validation dataset
so test dataset as a reality check
getting a test accuracy close to the validation set accuracy shows that we have not overfit
"""
test_loss,test_accuracy=model.evaluate(test_data)
print('\ntest loss==',test_loss)
print('test accuracy==',test_accuracy*100)




      1/Unknown - 2s 2s/step - loss: 0.1180 - accuracy: 0.9656
test loss== 0.11797527968883514
test accuracy== 96.56000137329102


In [21]:
test_data

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [24]:
for item in test_data:
    xi, yi = item
    pi = model.predict_on_batch(xi)
    print(xi["group"].shape, pi.shape)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'group'